In [1]:
import time
from queue import PriorityQueue
import heapq
import osmnx as ox
import networkx as nx

weight = "length"

In [2]:
G = ox.graph_from_place("Makati, Poblacion", network_type="drive")
nodes, edges = ox.graph_to_gdfs(G)

# NCR to test the Dijkstra
metro_G = ox.io.load_graphml(filepath="data/metro_drive.graphml")
metro_nodes, metro_edges = ox.graph_to_gdfs(metro_G)

In [3]:
test_orig = ox.nearest_nodes(metro_G, 120.99169, 14.51015)
test_dest = ox.nearest_nodes(metro_G, 121.0643, 14.6548)

In [4]:
print(f"Number of nodes: {len(metro_G.nodes)} and Number of edges: {len(metro_G.edges)}")

Number of nodes: 59055 and Number of edges: 148676


In [5]:
# Sample nodes
source = 32546634
target = 3650224015 # 248035797 

## Dijkstra test implementation

In [22]:
# Make path: list of node id's
def make_path(parent, goal):
    if goal not in parent:
        return None
    v = goal
    path = []
    while v is not None: # root has null parent
        path.append(v)
        v = parent[v]
    return path[::-1]

In [31]:
def make_coordinate_path(route):
    """Given a list of OSM node ids for the shortest path, return a list of the lat/long of each node"""
    path = []
    filtered_nodes = metro_nodes[metro_nodes.index.isin(route)]
    coordinates = filtered_nodes[['y', 'x']]
    for node in route:
        path.append(coordinates.loc[node].values.tolist())

    return path

In [7]:
# Taken from https://gist.github.com/qpwo/cda55deee291de31b50d408c1a7c8515 (just reworked the search on neighbors)
def dijkstra_queue(G, start, goal):
    """ Uniform-cost search / dijkstra """
    visited = set()
    cost = {start: 0}
    parent = {start: None}
    todo = PriorityQueue()
  
    todo.put((0, start))
    while todo:
        while not todo.empty():
            _, vertex = todo.get() # finds lowest cost vertex
            # loop until we get a fresh vertex
            if vertex not in visited: break
        else: # if todo ran out
            break # quit main loop
        visited.add(vertex)
        if vertex == goal:
            break
        
        # Explored the MultiDiGraph documentation from NetworkX, and managed to get the neighbor and edge lenghts using out_edges
        for _, neighbor, length in G.out_edges(vertex, data="length"):
            if neighbor in visited: continue
            old_cost = cost.get(neighbor, float('inf')) # default to infinity
            new_cost = cost[vertex] + length
            if new_cost < old_cost:
                todo.put((new_cost, neighbor))
                cost[neighbor] = new_cost
                parent[neighbor] = vertex

    return parent

In [8]:
# Instead of queue module, use heapq (see if any changes)
def dijkstra_heapq(G, start, goal):
    """ Uniform-cost search / dijkstra """
    visited = set()
    cost = {start: 0}
    parent = {start: None}
    todo = []
  
    heapq.heappush(todo, (0, start))
    while todo:
        while todo:
            _, vertex = heapq.heappop(todo) # finds lowest cost vertex
            # loop until we get a fresh vertex
            if vertex not in visited: break
        else: # if todo ran out
            break # quit main loop
        visited.add(vertex)
        if vertex == goal:
            break
        
        # Explored the MultiDiGraph documentation from NetworkX, and managed to get the neighbor and edge lenghts using out_edges
        for _, neighbor, length in G.out_edges(vertex, data="length"):
            if neighbor in visited: continue
            old_cost = cost.get(neighbor, float('inf')) # default to infinity
            new_cost = cost[vertex] + length
            if new_cost < old_cost:
                heapq.heappush(todo, (new_cost, neighbor))
                cost[neighbor] = new_cost
                parent[neighbor] = vertex

    return parent

In [9]:
# Simplified the while checks for already visited nodes into just ifs
def dijkstra_heapq_1(G, start, goal):
    """ Uniform-cost search / dijkstra """
    visited = set()
    cost = {start: 0}
    parent = {start: None}
    todo = [(0, start)]
  
    while todo:
        _, vertex = heapq.heappop(todo) # finds lowest cost vertex

        # Pop a different node from the queue when it has already been visited
        if vertex in visited:
            continue
        
        # End Dijkstra when goal node is visited 
        if vertex == goal:
            break

        visited.add(vertex)

        
        # Explored the MultiDiGraph documentation from NetworkX, and managed to get the neighbor and edge lenghts using out_edges
        for _, neighbor, length in G.out_edges(vertex, data="length"):
            if neighbor in visited:
                continue

            old_cost = cost.get(neighbor, float('inf')) # default to infinity
            new_cost = cost[vertex] + length

            if new_cost < old_cost:
                heapq.heappush(todo, (new_cost, neighbor))
                cost[neighbor] = new_cost
                parent[neighbor] = vertex

    return parent

## Dijkstra heapq, edge relaxation ignore checking of costs

Suppose that $v$ is a node we have just visited. The next step involves performing edge relaxation on each of its neighboring vertices. That is, given the tentative cost of $v$ and the edge length to its neighbor, we check if the sum of these two is better (i.e., less) than the tentative cost of the neighbor vertex.

**Not checking edge relaxation improvements works**

The usual way to do this is having an `old_cost` which is the tentative cost of the neighbor vertex and a `new_cost` which is the tentative cost of the visited/parent vertex plus the edge length to the neighbor: If `new_cost` is less than `old_cost`, then we *update* the neighbor vertex's tentative cost

But how did we avoid this checking condition in our code? Why does just adding the new tentative cost into the queue work without checking if it's an improvement? See that even if we have multiple costs for the same neighbor vertex in the queue, when it is time to pop that vertex, we will pop the minimum tentative cost from the queue out of all those multiple costs. And once we have popped that vertex, the other tentative costs for that vertex still in the queue will be disregarded by our `if` checks earlier

**Not initializing nodes with infinity works**

We did not initialize the tentative costs of the vertices with infinity since the first update we will perform for the tentative cost will invariably be added to the queue (in fact no matter what the tentative cost is we will always add it the to the queue, as per the previous paragraph)

**Adding the checks for the costs seems to be slower**

Just don't add the check



In [10]:
# Dijkstra heapq, no checks if new tentative cost is better than old tentative cost; just add all tentative costs to the queue
def dijkstra_heapq_no_check(G, source, target):
    visited = set()
    queue = [(0, source, None)]  # (tentative cost, vertex, parent)
    parents = {}
    
    while queue:
        vertex_cost, vertex, parent = heapq.heappop(queue)  # Get lowest cost vertex from the queue

        if vertex in visited:
            continue

        visited.add(vertex) # Consider vertex as visited
        parents[vertex] = parent
        
        # End algorithm when vertex visited is the goal node
        if vertex == target:
            break

        # Relax neighbor vertices of vertex
        for _, neighbor, edge_length in G.out_edges(vertex, data="length"):
            if neighbor in visited:
                continue

            # No need to check if new cost is an improvement, just add it to the queue; other costs will be disregarded later on
            neighbor_cost = vertex_cost + edge_length 
            heapq.heappush(queue, (neighbor_cost, neighbor, vertex))

    return parents


## Compare performance of Dijkstra implementation from osmNX/NetworkX

Noticed that the `nx.dijkstra_path` from NetworkX gives out a different path compared to osmNX (which uses a different Dijkstra function in the NetworkX library). But the `ox.shortest_path` route matches our implementation's route: all is good

In [23]:
# Run the Dijkstra with queue
start = time.perf_counter() 
parent_q = dijkstra_queue(metro_G, test_orig, test_dest)
test_route_q = make_path(parent_q, test_dest)
end = time.perf_counter() 

print(f"The time it took to run Dijkstra {end - start}")

The time it took to run Dijkstra 0.3785803999999189


In [24]:
# Run the Dijkstra from OSMnx
start = time.perf_counter() 
parent_o = ox.shortest_path(metro_G, test_orig, test_dest)
end = time.perf_counter() 

print(f"The time it took to run Dijkstra {end - start}")

The time it took to run Dijkstra 0.4400153999995382


In [36]:
# Run the Dijkstra from NetworkX bidirectional_search
start = time.perf_counter() 
parent_o = nx.bidirectional_dijkstra(metro_G, test_orig, test_dest, weight="length")
end = time.perf_counter() 

print(f"The time it took to run Dijkstra {end - start}")

The time it took to run Dijkstra 0.2340009999988979


In [34]:
# Run the Dijkstra with heapq
start = time.perf_counter() 
parent_h = dijkstra_heapq(metro_G, test_orig, test_dest)
test_route_h = make_path(parent_h, test_dest)
end = time.perf_counter() 

print(f"The time it took to run Dijkstra {end - start}")

The time it took to run Dijkstra 0.25858139999945706


In [39]:
# Run the Dijkstra with heapq no checks
start = time.perf_counter() 
parent_h_no_checks = dijkstra_heapq_no_check(metro_G, test_orig, test_dest)
test_route_h_no_check = make_path(parent_h_no_checks, test_dest)
coordinate_route = make_coordinate_path(test_route_h_no_check)
end = time.perf_counter() 

print(f"The time it took to run Dijkstra {end - start}")

The time it took to run Dijkstra 0.2693698000002769


In [28]:
# Test Dijkstra produced same route
if test_route_h_no_check == parent_o:
    print(test_route_h_no_check)

## Plot the route

In [30]:
nx_route = ox.routing.route_to_gdf(metro_G, nx_path, weight=weight)
m = nx_route.explore(color="skyblue", tiles="cartodbdarkmatter", style_kwds={"weight": 5, "opacity": 0.5})
m

NameError: name 'nx_path' is not defined

In [ ]:
# Plot the nodes and edges, and the shortest path route from our sample points
route = ox.shortest_path(metro_G, test_orig, test_dest) # To compare with osmnx shortest path
route_edges = ox.routing.route_to_gdf(metro_G, route, weight=weight)

test_route_edges = ox.routing.route_to_gdf(metro_G, test_route_h_no_check, weight=weight) # test_route holds our Dijktstra implementation's

m = test_route_edges.explore(color="skyblue", tiles="cartodbdarkmatter", style_kwds={"weight": 5, "opacity": 0.5})
route_edges.explore(m=m, color ="pink")

NameError: name 'test_route_h_no_check' is not defined